In [1]:
import sqlite3
import pandas as pd

This page includes the process in evaluating the result of an A/B testing. It contains the following sections:

1. Data Cleaning
3. Evaluate SRM and general metrics for A/AA groups
4. Evaluate A/B testing Result

# Data Cleaning

The primary goal of our data cleaning process is to isolate users who meet the criteria for inclusion in the experiment — the target audience (TA) users. We have defined selection criteria that ensure the users are appropriately qualified for the experiment's objectives. The criteria include:

1. **Event Timing**: The user's events should occur only after they have been recorded in the experiment tracking table. Additionally, the user registration date must be earlier than the event date recorded on the experiment table to ensure data consistency and reliability.

2. **Platform Specification**: We restrict our analysis to users on iOS platforms. This focus helps control for platform-specific variables and ensures that the user experience is consistent across the study group.

3. **Page Interaction**: It is essential that users have interacted with the "Trending" section on the home page. This interaction is crucial as it directly relates to the experiment's area of study, potentially affecting user behavior and engagement metrics.


In [4]:
# Load CSV files into Pandas
experiment_df = pd.read_csv('experiment_data/experiment_table.csv')
user_df = pd.read_csv('experiment_data/user_table.csv')
event_df = pd.read_csv('experiment_data/event_table.csv')

In [5]:
print(experiment_df.head())

   user_id  group_id  event_date device
0        1         0  2024-10-15    ios
1        2         2  2024-10-27    ios
2        3         1  2024-10-25    ios
3        4         2  2024-10-17    ios
4        7         0  2024-10-21    ios


We use SQL for data cleaning as it is commonly used tool

In [7]:
#Create SQLite in-memory database
conn = sqlite3.connect(":memory:")  # Uses in-memory storage (free and fast)
cursor = conn.cursor()

# Step 3: Store DataFrames as SQL tables
experiment_df.to_sql("experiment", conn, index=False, if_exists="replace")
user_df.to_sql("user", conn, index=False, if_exists="replace")
event_df.to_sql("event", conn, index=False, if_exists="replace")

# Step 4: Run SQL queries for data cleaning (example: removing duplicates)
TA_df = pd.read_sql_query("""

with TA as (
    select 
        distinct e.user_id
    from event e 
    left join experiment exp
            on e.user_id = exp.user_id 
            and date(e.timestamp) >= exp.event_date
            and e.device = exp.device 
    left join user u
        on e.user_id = u.user_id
        and exp.event_date >= u.registration_date
    where 
        event_name = 'view_section'
        and page_name = 'home'
        and section_name = 'trending'
        and exp.device = 'ios'
)

select 
    e.*,
    exp.group_id,
    exp.event_date
from event e 
left join experiment exp
    on e.user_id = exp.user_id 
    and date(e.timestamp) >= exp.event_date
    and e.device = exp.device     
where e.user_id in (select * from TA)

""", conn)

# Step 5: Retrieve and process cleaned data
print(TA_df.head())

# Close connection when done
conn.close()

   user_id device            timestamp    event_name page_name section_name  \
0        1    ios  2024-10-15 19:37:00     view_page      home         None   
1        1    ios  2024-10-15 19:37:01  view_section      home    discovery   
2        1    ios  2024-10-15 19:41:07    click_item      home    discovery   
3        1    ios  2024-10-15 19:45:31  view_section      home     trending   
4        1    ios  2024-10-15 19:50:01    click_item      home     trending   

   group_id  event_date  
0         0  2024-10-15  
1         0  2024-10-15  
2         0  2024-10-15  
3         0  2024-10-15  
4         0  2024-10-15  


# SRM and Comparing General Metric

## SRM
Checking SRM after 3 days running an experiment

In [10]:
from test_funcs import check_srm
result = check_srm(n_AA=TA_df[(TA_df['event_date']<='2024-10-07')& (TA_df['group_id']==0)]['user_id'].nunique(), 
                   n_A=TA_df[(TA_df['event_date']<='2024-10-07')& (TA_df['group_id']==1)]['user_id'].nunique(), 
                   n_B=TA_df[(TA_df['event_date']<='2024-10-07')& (TA_df['group_id']==2)]['user_id'].nunique(), 
                   ratio_AA=0.25, ratio_A=0.25, ratio_B=0.5)  # Replace with real numbers
print(result)

{'Observed Counts': {'AA': 3008, 'A': 2947, 'B': 6197}, 'Expected Counts': {'AA': 3038.0, 'A': 3038.0, 'B': 6076.0}, 'Chi-Square Statistic': 5.431698485845951, 'p-value': 0.06614875278646357, 'SRM Detected': False}


## Check for general metrics between A and AA group

In [12]:
# Step 2: Create SQLite in-memory database
conn = sqlite3.connect(":memory:")  # Uses in-memory storage (free and fast)
cursor = conn.cursor()

# Step 3: Store DataFrames as SQL tables
TA_df.to_sql("ta_df", conn, index=False, if_exists="replace")

# Step 4: Run SQL queries for data cleaning (example: removing duplicates)
general_metric = pd.read_sql_query("""

    select 
        group_id,
        count(distinct 
            case when event_name = 'click_item' and page_name ='home' and section_name ='discovery'
                then user_id
            else null end) as num_user_click,
        count(distinct 
            case when event_name = 'view_section' and page_name ='home' and section_name ='discovery'
                then user_id
            else null end) as num_user_exposed
    from ta_df 
    group by 1

""", conn)

# Step 5: Retrieve and process cleaned data
print(general_metric.head())

# Close connection when done
conn.close()

   group_id  num_user_click  num_user_exposed
0         0            7568             24710
1         1            7510             24622
2         2           15184             50668


In [13]:
from test_funcs import z_test_proportions

result = z_test_proportions(general_metric[general_metric['group_id']==0]['num_user_click'].sum(),
                            general_metric[general_metric['group_id']==0]['num_user_exposed'].sum(), 
                            general_metric[general_metric['group_id']==1]['num_user_click'].sum(), 
                            general_metric[general_metric['group_id']==1]['num_user_exposed'].sum())
print(result)

{'CTR_A': 0.3062727640631323, 'CTR_B': 0.30501177808463975, 'CTR_Difference': -0.001260985978492557, 'Z-score': -0.3039800897510471, 'p-value': 0.7611430599205433, '95% CI': (-0.00939141026016016, 0.006869438303175046), 'is_significant': False}


# A/B Testing Result

In [15]:
# Step 2: Create SQLite in-memory database
conn = sqlite3.connect(":memory:")  # Uses in-memory storage (free and fast)
cursor = conn.cursor()

# Step 3: Store DataFrames as SQL tables
TA_df.to_sql("ta_df", conn, index=False, if_exists="replace")

# Step 4: Run SQL queries for data cleaning (example: removing duplicates)
success_metric = pd.read_sql_query("""

    select 
        case when group_id ==0 then 1 else group_id end as group_id,
        count(distinct 
            case when event_name = 'click_item' and page_name ='home' and section_name ='trending'
                then user_id
            else null end) as num_user_click,
        count(distinct 
            case when event_name = 'view_section' and page_name ='home' and section_name ='trending'
                then user_id
            else null end) as num_user_exposed
    from ta_df 
    group by 1

""", conn)

# Step 5: Retrieve and process cleaned data
print(success_metric.head())

# Close connection when done
conn.close()

   group_id  num_user_click  num_user_exposed
0         1           14694             49332
1         2           20376             50668


In [16]:
result = z_test_proportions(success_metric[success_metric['group_id']==1]['num_user_click'].sum(),
                            success_metric[success_metric['group_id']==1]['num_user_exposed'].sum(), 
                            success_metric[success_metric['group_id']==2]['num_user_click'].sum(), 
                            success_metric[success_metric['group_id']==2]['num_user_exposed'].sum())
print(result)

{'CTR_A': 0.29785940160544877, 'CTR_B': 0.4021473119128444, 'CTR_Difference': 0.10428791030739565, 'Z-score': 34.55211394502931, 'p-value': 0.0, '95% CI': (0.09837219271770982, 0.11020362789708148), 'is_significant': True}


**The new UI design for the trending section has a positive and significant impact on the click-through rate, increasing it by approximately 10.43%.** This improvement suggests that the redesigned trending section is more effective at attracting users and encouraging them to engage with the displayed items.